In [7]:
import os
if not os.path.isdir("twitch_egos"):
    os.system("unzip ../twitch_egos.zip")

Archive:  ../twitch_egos.zip
   creating: twitch_egos/
  inflating: twitch_egos/README.txt  
  inflating: twitch_egos/citing.txt  
  inflating: twitch_egos/twitch_target.csv  
  inflating: twitch_egos/twitch_edges.json  


In [8]:
import json
from tqdm import tqdm
import pandas as pd
import numpy as np

with open("twitch_egos/twitch_edges.json") as fin:
    edges_in: dict = json.load(fin)
label_df = pd.read_csv("twitch_egos/twitch_target.csv")
label_df


,id,target
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0
...,...,...
127089,127089,1
127090,127090,1
127091,127091,1
127092,127092,1


In [9]:
# put all the edges of separate graphs into one graph
edges = []
users = []
friends = []
labels = []
d_type = {}
offset = 0
max_id = -1
for k, v in tqdm(edges_in.items()):
    for e in v:
        e[0] += offset
        e[1] += offset
        max_id = max([max_id, e[0], e[1]])
    offset = max_id + 1
    edges.extend(v)
    current_verts = list(dict.fromkeys([l for x in v for l in x]))
    users.append(current_verts[0])
    labels.append(label_df.iloc[int(k)].target)
    friends.extend(current_verts[1:])

d_type.update({k:'User' for k in users})
d_type.update({k:"Friend" for k in friends})
print(f"{len(labels)= }")
print(f"{len(labels)==len(users) = }")
print(len(users))
print(edges[:5], edges[-5:])
users[:5], users[-5:]


100%|██████████| 127094/127094 [00:08<00:00, 15369.29it/s]


len(labels)= 127094
len(labels)==len(users) = True
127094
[[35, 0], [35, 1], [35, 2], [35, 3], [35, 4]] [[3771070, 3771078], [3771068, 3771078], [3771066, 3771078], [3771058, 3771079], [3771069, 3771079]]


([35, 76, 108, 146, 168], [3770960, 3770995, 3771036, 3771055, 3771080])

In [10]:
print(friends[0], users[0])
assert d_type[friends[0]] == "Friend"
assert d_type[users[0]] == "User"

0 35


In [11]:
edges_typed = []
for e in tqdm(edges):
    edges_typed.append([e[0],d_type[e[0]],e[1],d_type[e[1]]])
edges_typed[:3], edges_typed[-3:]

100%|██████████| 18367047/18367047 [00:24<00:00, 758555.94it/s] 


([[35, 'User', 0, 'Friend'],
  [35, 'User', 1, 'Friend'],
  [35, 'User', 2, 'Friend']],
 [[3771066, 'Friend', 3771078, 'Friend'],
  [3771058, 'Friend', 3771079, 'Friend'],
  [3771069, 'Friend', 3771079, 'Friend']])

In [12]:
assert len(users) == 127_094
assert len(friends) + 127_094 == 3_771_081, len(friends)+ len(users)
assert len(edges_typed) == 18_367_047, len(edges_typed)


In [13]:
df = pd.DataFrame({"id": users, "label":labels})
df = df.astype(int)
df.to_csv("twitch_egos/Twitch_users.csv", index=False)
print(df.shape)
df


(127094, 2)


,id,label
0,35,0
1,76,0
2,108,0
3,146,1
4,168,0
...,...,...
127089,3770960,1
127090,3770995,1
127091,3771036,1
127092,3771055,1


In [14]:
df = pd.DataFrame({"id": friends})
df = df.astype(int)
df.to_csv("twitch_egos/Twitch_friends.csv", index=False)
print(df.shape)
df


(3643987, 1)


,id
0,0
1,1
2,2
3,3
4,4
...,...
3643982,3771075
3643983,3771076
3643984,3771077
3643985,3771078


In [15]:
edges = np.array(edges_typed)
edf = pd.DataFrame({"src": edges[:, 0], "src_type":edges[:,1], "tgt": edges[:, 2], "tgt_type":edges[:,3]})
edf.to_csv("twitch_egos/Twitch_edges.csv", index=False)
edf


,src,src_type,tgt,tgt_type
0,35,User,0,Friend
1,35,User,1,Friend
2,35,User,2,Friend
3,35,User,3,Friend
4,35,User,4,Friend
...,...,...,...,...
18367042,3771070,Friend,3771078,Friend
18367043,3771068,Friend,3771078,Friend
18367044,3771066,Friend,3771078,Friend
18367045,3771058,Friend,3771079,Friend


In [18]:
vm_name = input('Where is the data going? > ')
pth = input('path to the data dir')
!scp twitch_egos/Twitch_users.csv $vm_name:$pth
!scp twitch_egos/Twitch_friends.csv $vm_name:$pth
!scp twitch_egos/Twitch_edges.csv $vm_name:$pth

*******************************************************************************
*                                                                             *
*  This is a private computer system containing confidential information.     *
*  Any unauthorized attempt to access or use this computer system or any      *
*  information on it by employees or other persons may result in termination  *
*  of employment, civil fines, and criminal penalties.  This system must be   *
*  used for authorized business purposes only.                                *
*                                                                             *
*******************************************************************************
Twitch_users.csv                              100% 1205KB   1.1MB/s   00:01    
*******************************************************************************
*                                                                             *
*  This is a private computer system con